In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv( "../data/processed/train_1.csv")
test = pd.read_csv("../data/processed/test_1.csv")
validation = pd.read_csv("../data/processed/validation_1.csv")

In [3]:
from sklearn.model_selection import train_test_split
X = train['review']
y = train['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [4]:
from gensim.models import KeyedVectors
id_model = KeyedVectors.load_word2vec_format('../data/external/wiki.id.vec')
words = []
for word in id_model.vocab:
    words.append(word)

# Printing out number of tokens available
print("Number of Tokens: {}".format(len(words)))

# Printing out the dimension of a word vector 
print("Dimension of a word vector: {}".format(
    len(id_model[words[0]])
))

# Print out the vector of a word 
print("Vector components of a word: {}".format(
    id_model[words[0]]
))

Number of Tokens: 300686
Dimension of a word vector: 300
Vector components of a word: [-0.020878  -0.098768   0.010647   0.029345  -0.066832   0.20484
 -0.12876   -0.041663   0.15152    0.10408   -0.25066    0.18666
 -0.037608  -0.31225   -0.1883    -0.13681   -0.17851   -0.20356
 -0.28021   -0.01756    0.07171    0.11328   -0.22029   -0.083482
 -0.19687    0.056111  -0.11775    0.062158  -0.085196   0.20082
  0.57581   -0.083127   0.018557  -0.12359   -0.38605   -0.30369
  0.042381  -0.086606   0.021943   0.031174   0.18783    0.27425
 -0.16156    0.42373    0.24293    0.19421    0.56379   -0.042846
  0.0037499 -0.035674   0.11309    0.22571   -0.37663   -0.051349
 -0.062389  -0.048832   0.26946    0.19802   -0.3773     0.0048868
  0.17017   -0.042761   0.13404    0.15675   -0.12758   -0.22971
 -0.1304     0.33002    0.18366   -0.0072019 -0.15981   -0.41593
 -0.054073   0.0021753 -0.066927   0.16821   -0.1285    -0.19194
 -0.28232   -0.30573    0.26306   -0.099999   0.14894    0.22159

In [5]:
embedding = np.array([])
limit = 500
vector_dim = 300

In [6]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
i = 0
for word in id_model.vocab:
    # Break the loop if limit exceeds 
    if i == limit: break

    # Getting token 
    words.append(word)

    # Appending the vectors 
    embedding = np.append(embedding, id_model[word])

    i += 1

# Reshaping the embedding vector 
embedding = embedding.reshape(limit, vector_dim)


def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
#     assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')
    plt.savefig(filename)


# Creating the tsne plot [Warning: will take time]
tsne = TSNE(perplexity=30.0, n_components=2, init='pca', n_iter=5000)

low_dim_embedding = tsne.fit_transform(embedding)

# Finally plotting and saving the fig 
plot_with_labels(low_dim_embedding, words)

IndexError: index 500 is out of bounds for axis 0 with size 500

In [5]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
%time forest = forest.fit(train_data_features, y_train)

Training the random forest...
CPU times: user 1min 1s, sys: 133 ms, total: 1min 1s
Wall time: 1min 2s


In [6]:
test_data_features = vectorizer.transform(X_test)
test_data_features = test_data_features.toarray()
pred = forest.predict(test_data_features)

In [7]:
from sklearn.metrics import accuracy_score, log_loss,confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
acc = accuracy_score(y_test,pred)
cm = confusion_matrix(y_test,pred)
print("Accuracy Score: " + str(acc))
print("Confusion Matrix: "+ str(cm))

Accuracy Score: 0.7710427135678392
Confusion Matrix: [[ 579  497]
 [ 232 1876]]


In [8]:
# 1. import
from sklearn.naive_bayes import MultinomialNB

# 2. instantiate a Multinomial Naive Bayes model
nb = MultinomialNB()
%time nb.fit(train_data_features, y_train)
pred = nb.predict(test_data_features)

CPU times: user 145 ms, sys: 3.97 ms, total: 149 ms
Wall time: 148 ms


In [9]:
acc = accuracy_score(y_test, pred)
print("Accuracy Score: " + str(acc))

Accuracy Score: 0.7584798994974874


In [10]:
null_ = []
for i in range(0,len(y_test)):
    null_.append(1)
null_accuracy = accuracy_score(y_test, null_)
print('Null accuracy:', null_accuracy)

Null accuracy: 0.6620603015075377


In [11]:
# 1. import
import lightgbm as lgb

In [12]:
# 2. instantiate a Multinomial Naive Bayes model
lgbm = lgb.LGBMClassifier()
%time lgbm.fit(train_data_features, y_train)
pred = lgbm.predict(test_data_features)
acc = accuracy_score(y_test, pred)
print("Accuracy Score: " + str(acc))

CPU times: user 5.88 s, sys: 27.8 ms, total: 5.91 s
Wall time: 1.6 s
Accuracy Score: 0.7751256281407035


/home/nakama/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [13]:
import xgboost as xgb

In [14]:
xgbo = xgb.XGBClassifier()
%time xgbo.fit(train_data_features, y_train)
predic = xgbo.predict(test_data_features)
acc = accuracy_score(y_test, predic)
print("Accuracy Score: " + str(acc))

CPU times: user 1min 29s, sys: 296 ms, total: 1min 29s
Wall time: 1min 29s
Accuracy Score: 0.7550251256281407


/home/nakama/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [15]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
%time logistic.fit(train_data_features, y_train)
predic = logistic.predict(test_data_features)
acc = accuracy_score(y_test, predic)
print("Accuracy Score: " + str(acc))

CPU times: user 157 ms, sys: 5 µs, total: 157 ms
Wall time: 157 ms
Accuracy Score: 0.7713567839195979
